In [3]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 

In [6]:
from dataclasses import dataclass
from datetime import datetime, time
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [15]:
from dataclasses import dataclass
@dataclass
class RawMovie:
    name: str
    url: str

@dataclass
class Screening:
    date: datetime
    threatre: str
@dataclass
class Movie(RawMovie):
    screening: list[Screening]

In [14]:
base_url = "https://hkmovie6.com/"
url = "https://hkmovie6.com/showing"
headers = {
    'Cookie': 'lang=enGB;'
}
# Send a GET request to the URL
response = requests.get(url,headers=headers)
if response.status_code != 200:
    print(f"Failed to fetch the page. Status code: {response.status_code}")
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')


# Find all div elements with class starting with 'name'
movie_name_divs = soup.find_all('div', class_=lambda x: x and x.startswith('movie show'))
RawMovies: list[RawMovie] = []
now = datetime.now()
# Extract and print the movie names
for name_div in movie_name_divs:
  href = name_div.find_next('a')['href'] if name_div.find('a') else None
  name = name_div.find_next('div', class_=lambda x: x and x.startswith('name')).get_text(strip=True)
  print(f"{name} - {href}")
  RawMovies.append(RawMovie(name=name,url=href))


The Apprentice - /movie/ab8fa39d-15c6-4bab-a546-c07205718838
𝗺𝟲 𝘀𝗰𝗿𝗲𝗲𝗻𝗶𝗻𝗴 𝟬𝟭𝟲 ➤ Strange Darling Special Screening - /movie/16028424-e2d4-4aca-8129-d057ded1e906
𝗺𝟲 𝘀𝗰𝗿𝗲𝗲𝗻𝗶𝗻𝗴 𝟬𝟭𝟳 ➤ Papa Special Screening - /movie/58feac2d-d956-422d-b42b-79932f852ad3
Blossoms Under Somewhere - /movie/b113dadb-3011-466b-8945-20fb2f855fe8
The Last Dance - /movie/ed85c5dc-7853-4d13-bc3e-7c082b2baf13
Cesium Fallout - /movie/e1a368cc-db87-4be1-be32-821a1d497115
Wicked - /movie/58afc3c0-d791-46af-ba1a-e54b1bcc77a8
m6 keylight | 電影專題 - https://www.instagram.com/p/DCg0TJKPnHV/?img_index=1
An Abandoned Team - /movie/d10a3646-c82d-43de-8850-54fba504f788
Gladiator II - /movie/f2b56b0b-a6d4-4b04-afa4-ea5c30c5c60e
Ghost Cat Anzu - /movie/cfb8a347-ff3f-49a4-9d6d-d02e9abd9442
Red One - /movie/6d7d2022-d2e6-4dbc-b971-be1ed4d0414b
Weekend in Taipei - /movie/b6071de7-808c-4a93-a211-0d7f911f02e9
OVERLORD: The Sacred Kingdom - /movie/e8713d9a-6fb6-4404-8b9f-270dc3c50450
BAEKHYUN: Lonsdaleite [dot] IN CINEMAS - /movie/9cd2972

In [18]:
chrome_options = Options() 
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=chrome_options)
for rawMovie in RawMovies:
  url = "https://hkmovie6.com" + rawMovie.url + "/showtime"
  try:
    driver.get(url)
  except:
    continue
  date = driver.find_element(By.CLASS_NAME, "date").text
  (day, month) = date.split("/")
  if not now.month == int(month) or not now.day == int(day):
    print("Not today")
    continue
  cinemas_list = driver.find_elements(By.CLASS_NAME, "cinemas")
  if len(cinemas_list) == 0:
    continue
  cinemas_divs = driver.find_elements(By.CLASS_NAME, "cinemas")[0].find_elements(By.CLASS_NAME, "cinema")
  screenings = []
  for cinemas_div in cinemas_divs:
    threatre = cinemas_div.find_element(By.CLASS_NAME, "cinemaName").text
    time_divs = cinemas_div.find_elements(By.CLASS_NAME, "time")
    for time_div in time_divs:
      try:
        if len(time_div.text.split(":")) == 0:
          continue 
        screening = Screening(date=now.replace(hour=int(time_div.text.split(":")[0]),minute=int(time_div.text.split(":")[1]),second=0),threatre=threatre)
        screenings.append(screening)
      except:
        continue
  movie = Movie(name=rawMovie.name,url=rawMovie.url,screening=screenings)
  print(f"{movie.name} - {movie.screening}")

driver.quit()

The Apprentice - [Screening(date=datetime.datetime(2024, 11, 27, 19, 30, 0, 409836), threatre='高先電影院'), Screening(date=datetime.datetime(2024, 11, 27, 21, 40, 0, 409836), threatre='高先電影院'), Screening(date=datetime.datetime(2024, 11, 27, 20, 0, 0, 409836), threatre='PREMIERE ELEMENTS'), Screening(date=datetime.datetime(2024, 11, 27, 16, 35, 0, 409836), threatre='影滙戲院'), Screening(date=datetime.datetime(2024, 11, 27, 19, 10, 0, 409836), threatre='影滙戲院'), Screening(date=datetime.datetime(2024, 11, 27, 19, 20, 0, 409836), threatre='影滙戲院'), Screening(date=datetime.datetime(2024, 11, 27, 19, 20, 0, 409836), threatre='CANDY PARK BY CINEMA CITY (愉景新城)'), Screening(date=datetime.datetime(2024, 11, 27, 20, 30, 0, 409836), threatre='Lumen Cinema'), Screening(date=datetime.datetime(2024, 11, 27, 19, 10, 0, 409836), threatre='博納國際影城澳門金銀島店')]
Not today
Not today
Cesium Fallout - [Screening(date=datetime.datetime(2024, 11, 27, 22, 0, 0, 409836), threatre='英皇戲院（銅鑼灣時代廣場）'), Screening(date=datetime.date